This one can be deleted

In [3]:
import os
import numpy as np
import cv2
import joblib
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Define the classes
classes = ["Defective", "Non defective"]

# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Load the trained logistic regression model
model_filename = '/Users/janekkorczynski/Desktop/BizHack/logistic_regression_model.joblib'
clf = joblib.load(model_filename)
print(f"Model loaded from {model_filename}")

def preprocess_frame(frame):
    img = Image.fromarray(frame).convert('RGB')
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def detect_anomalies(frame):
    img_array = preprocess_frame(frame)
    feature = base_model.predict(img_array).flatten().reshape(1, -1)
    prediction = clf.predict(feature)
    confidence = clf.predict_proba(feature)
    return prediction[0], confidence[0]

def draw_bounding_box(frame, box, label, confidence):
    (startX, startY, endX, endY) = box
    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
    text = f"{label}: {confidence:.2f}"
    y = startY - 10 if startY - 10 > 10 else startY + 10
    cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        prediction, confidence = detect_anomalies(frame)

        if classes[prediction] == "Defective":
            # Assume that we have bounding box coordinates for anomalies
            # For demonstration, we use a static box
            box = (50, 50, 200, 200)
            frame = draw_bounding_box(frame, box, classes[prediction], confidence[prediction])

        out.write(frame)
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

input_video_path = '/Users/janekkorczynski/Desktop/BizHack/recordings/Defect/Screen Recording 2024-07-24 at 22.49.04.mp4'
output_video_path = 'output_video.avi'
process_video(input_video_path, output_video_path)


Model loaded from /Users/janekkorczynski/Desktop/BizHack/logistic_regression_model.joblib
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/st

This one is juts for a video generation

In [1]:
import os
import numpy as np
import cv2
import torch
from PIL import Image

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.25  # confidence threshold

def preprocess_frame(frame):
    img = Image.fromarray(frame)
    return img

def detect_anomalies(frame):
    img = preprocess_frame(frame)
    results = model(img)
    return results

def draw_bounding_box(frame, bbox, label, confidence):
    x1, y1, x2, y2 = bbox
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    text = f"{confidence:.2f}"
    y = y1 - 10 if y1 - 10 > 10 else y1 + 10
    cv2.putText(frame, text, (x1, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = detect_anomalies(frame)

        for detection in results.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = detection
            label = model.names[int(cls)]
            frame = draw_bounding_box(frame, (int(x1), int(y1), int(x2), int(y2)), label, conf)

        out.write(frame)
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

input_video_path = '/Users/janekkorczynski/Desktop/BizHack/recordings/Defect/Screen Recording 2024-07-24 at 22.49.04.mp4'
output_video_path = 'output_video.mp4'
process_video(input_video_path, output_video_path)


Using cache found in /Users/janekkorczynski/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-24 Python-3.9.18 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Can be deleted

In [2]:
import os
import numpy as np
import cv2
import torch
from PIL import Image

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.25  # confidence threshold

def preprocess_frame(frame):
    img = Image.fromarray(frame)
    return img

def detect_anomalies(frame):
    img = preprocess_frame(frame)
    results = model(img)
    return results

def draw_bounding_box(frame, bbox, label, confidence):
    x1, y1, x2, y2 = bbox
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    text = f"{confidence:.2f}"
    y = y1 - 10 if y1 - 10 > 10 else y1 + 10
    cv2.putText(frame, text, (x1, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = detect_anomalies(frame)

        for detection in results.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = detection
            label = model.names[int(cls)]
            frame = draw_bounding_box(frame, (int(x1), int(y1), int(x2), int(y2)), label, conf)

        out.write(frame)
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

input_video_path = '/Users/janekkorczynski/Desktop/BizHack/recordings/Defect/Screen Recording 2024-07-24 at 23.50.02.mp4'
output_video_path = 'output_video.avi'
process_video(input_video_path, output_video_path)


Using cache found in /Users/janekkorczynski/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-24 Python-3.9.18 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


: 

HERE STARTS TEH PROPER CODE 

In [19]:
import os
import numpy as np
import cv2
import torch
from PIL import Image

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.25  # confidence threshold

def preprocess_frame(frame):
    img = Image.fromarray(frame)
    return img

def detect_anomalies(frame):
    img = preprocess_frame(frame)
    results = model(img)
    return results

def draw_bounding_box(frame, bbox, label, confidence):
    x1, y1, x2, y2 = bbox
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    text = f"{confidence:.2f}"
    y = y1 - 10 if y1 - 10 > 10 else y1 + 10
    cv2.putText(frame, text, (x1, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # List to store bounding box coordinates
    bbox_coords = []
    screenshot_folder = 'vid_photos'
    os.makedirs(screenshot_folder, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = detect_anomalies(frame)

        for detection in results.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = detection
            label = model.names[int(cls)]
            frame = draw_bounding_box(frame, (int(x1), int(y1), int(x2), int(y2)), label, conf)
            # Save the coordinates and confidence
            bbox_coords.append({'frame': int(cap.get(cv2.CAP_PROP_POS_FRAMES)), 
                                'confidence': conf, 
                                'bbox': [int(x1), int(y1), int(x2), int(y2)]})
            # Save screenshot
            screenshot_path = os.path.join(screenshot_folder, f"frame_{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}.jpg")
            cv2.imwrite(screenshot_path, frame)

        out.write(frame)
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save the bounding box coordinates to a file in the screenshot folder
    output_coords_path = os.path.join(screenshot_folder, 'bounding_box_coords.txt')
    with open(output_coords_path, 'w') as f:
        for bbox in bbox_coords:
            f.write(f"Frame {bbox['frame']}, Confidence: {bbox['confidence']:.2f}, BBox: {bbox['bbox']}\n")

input_video_path = '/Users/janekkorczynski/Desktop/BizHack/recordings/Defect/Screen Recording 2024-07-24 at 22.49.04.mp4'
output_video_path = 'output_video.mp4'
process_video(input_video_path, output_video_path)


Using cache found in /Users/janekkorczynski/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-24 Python-3.9.18 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [20]:
import os
import numpy as np
import joblib
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Define the classes
classes = ["Defective", "Non defective"]

# Load the ResNet50 model without the top layer for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Load the trained logistic regression model
model_filename = 'logistic_regression_model.joblib'
clf = joblib.load(model_filename)
print(f"Model loaded from {model_filename}")

def preprocess_frame(frame_path):
    img = Image.open(frame_path).convert('RGB')
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def classify_frame(frame_path):
    img_array = preprocess_frame(frame_path)
    feature = base_model.predict(img_array).flatten().reshape(1, -1)
    prediction = clf.predict(feature)
    confidence = clf.predict_proba(feature)
    return prediction[0], confidence[0]

def classify_saved_screenshots(screenshot_folder):
    defect_count = 0
    total_frames = 0

    for filename in os.listdir(screenshot_folder):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(screenshot_folder, filename)
            prediction, confidence = classify_frame(frame_path)
            total_frames += 1
            if classes[prediction] == "Defective":
                defect_count += 1

            # Debug: print classification results
            print(f"{filename}: {classes[prediction]} ({confidence[prediction]:.2f})")

            # Optionally, save the frame with annotation
            if classes[prediction] == "Defective":
                annotated_frame = cv2.imread(frame_path)
                text = f"{classes[prediction]}: {confidence[prediction]:.2f}"
                cv2.putText(annotated_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.imwrite(os.path.join(screenshot_folder, f"defective_{filename}"), annotated_frame)

    defect_percentage = (defect_count / total_frames) * 100 if total_frames > 0 else 0
    print(f"Total Frames Analyzed: {total_frames}")
    print(f"Defective Frames: {defect_count}")
    print(f"Defective Percentage: {defect_percentage:.2f}%")

screenshot_folder = '/Users/janekkorczynski/Desktop/BizHack/vid_photos'
classify_saved_screenshots(screenshot_folder)


Model loaded from logistic_regression_model.joblib
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
frame_239.jpg: Defective (0.93)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
frame_263.jpg: Defective (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
frame_275.jpg: Non defective (0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
frame_249.jpg: Non defective (0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
frame_248.jpg: Non defective (0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
frame_274.jpg: Non defective (0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
frame_264.jpg: Defective (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
frame_270.jpg: Non defective (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
frame_265.jpg: Defective (0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
frame_273.jpg: Non defective (0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
frame_267.jpg: Non defective (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
frame_266.jpg: Non defective (1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
frame_109.jpg: Non defective (0.88)
1/

In [2]:
!pip3 install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 39.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.9 MB/s eta 0:00:00a 0:00:01
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 27.9 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import cv2
import torch
import joblib
import streamlit as st
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Define the classes
classes = ["Defective", "Non defective"]

# Load the YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.conf = 0.25  # confidence threshold

# Load the ResNet50 model without the top layer for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Load the trained logistic regression model
model_filename = 'logistic_regression_model.joblib'
clf = joblib.load(model_filename)
print(f"Model loaded from {model_filename}")

def preprocess_frame_for_yolo(frame):
    img = Image.fromarray(frame)
    return img

def detect_anomalies(frame):
    img = preprocess_frame_for_yolo(frame)
    results = yolo_model(img)
    return results

def draw_bounding_box(frame, bbox, confidence):
    x1, y1, x2, y2 = bbox
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    text = f"{confidence:.2f}"
    y = y1 - 10 if y1 - 10 > 10 else y1 + 10
    cv2.putText(frame, text, (x1, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    return frame

def preprocess_frame_for_classification(frame_path):
    img = Image.open(frame_path).convert('RGB')
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def classify_frame(frame_path):
    img_array = preprocess_frame_for_classification(frame_path)
    feature = base_model.predict(img_array).flatten().reshape(1, -1)
    prediction = clf.predict(feature)
    confidence = clf.predict_proba(feature)
    return prediction[0], confidence[0]

def process_video_and_save_screenshots(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # List to store bounding box coordinates
    bbox_coords = []
    screenshot_folder = 'vid_photos'
    os.makedirs(screenshot_folder, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = detect_anomalies(frame)

        for detection in results.xyxy[0].cpu().numpy():
            x1, y1, x2, y2, conf, cls = detection
            frame = draw_bounding_box(frame, (int(x1), int(y1), int(x2), int(y2)), conf)
            # Save the coordinates and confidence
            bbox_coords.append({'frame': int(cap.get(cv2.CAP_PROP_POS_FRAMES)), 
                                'confidence': conf, 
                                'bbox': [int(x1), int(y1), int(x2), int(y2)]})
            # Save screenshot
            screenshot_path = os.path.join(screenshot_folder, f"frame_{int(cap.get(cv2.CAP_PROP_POS_FRAMES))}.jpg")
            cv2.imwrite(screenshot_path, frame)

        out.write(frame)
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save the bounding box coordinates to a file in the screenshot folder
    output_coords_path = os.path.join(screenshot_folder, 'bounding_box_coords.txt')
    with open(output_coords_path, 'w') as f:
        for bbox in bbox_coords:
            f.write(f"Frame {bbox['frame']}, Confidence: {bbox['confidence']:.2f}, BBox: {bbox['bbox']}\n")

def classify_saved_screenshots(screenshot_folder):
    defect_count = 0
    total_frames = 0

    for filename in os.listdir(screenshot_folder):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(screenshot_folder, filename)
            prediction, confidence = classify_frame(frame_path)
            total_frames += 1
            if classes[prediction] == "Defective":
                defect_count += 1

            # Debug: print classification results
            print(f"{filename}: {classes[prediction]} ({confidence[prediction]:.2f})")

            # Optionally, save the frame with annotation
            if classes[prediction] == "Defective":
                annotated_frame = cv2.imread(frame_path)
                text = f"{classes[prediction]}: {confidence[prediction]:.2f}"
                cv2.putText(annotated_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.imwrite(os.path.join(screenshot_folder, f"defective_{filename}"), annotated_frame)

    defect_percentage = (defect_count / total_frames) * 100 if total_frames > 0 else 0
    st.write(f"Total Frames Analyzed: {total_frames}")
    st.write(f"Defective Frames: {defect_count}")
    st.write(f"Defective Percentage: {defect_percentage:.2f}%")

# Streamlit UI
st.title('Railway Defect Detection')
st.write('Upload a video file to analyze for defects.')

uploaded_file = st.file_uploader("Choose a video file...", type=["mp4", "avi", "mov"])

if uploaded_file is not None:
    
    # Create the uploads directory if it doesn't exist
    os.makedirs("uploads", exist_ok=True)
    
    input_video_path = os.path.join("uploads", uploaded_file.name)
    output_video_path = "output_video.mp4"

    # Save the uploaded file
    with open(input_video_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    
    st.write("Analyzing the video...")
    process_video_and_save_screenshots(input_video_path, output_video_path)
    
    st.write("Classifying the saved screenshots...")
    classify_saved_screenshots('vid_photos')
    
    st.write("Analysis complete. The results are displayed above.")


Using cache found in /Users/janekkorczynski/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-24 Python-3.9.18 torch-2.3.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Model loaded from logistic_regression_model.joblib
